In [0]:
from google.colab import drive
drive.mount('/content/drive/')
current_path = ''

import torch
import torch.nn as nn
import numpy as np
import torchvision
import torchvision.transforms as transforms

In [0]:
# helper function for computing accuracy
def get_acc(pred, y):
    pred = pred.float()
    y = y.float()
    return (y==pred).sum().float()/y.size(0)*100.

In [5]:
# Preprocessing steps on the training/testing data
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# download the dataset
trainset = torchvision.datasets.CIFAR10(root=current_path + '/data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(root=current_path + '/data', train=False, download=True, transform=transform_test)

Files already downloaded and verified
Files already downloaded and verified


In [0]:
def get_model_acc(model, loader):
    ys = []
    y_preds = []
    for x, y in loader:
        ys.append(y)
        y_preds.append(torch.argmax(model(x), dim=1))
    y = torch.cat(ys, dim=0)
    y_pred = torch.cat(y_preds, dim=0)
    print((y == y_pred).sum())
    return get_acc(y_pred, y)

In [0]:
import torch.nn.functional as F
class CNNClassifier(nn.Module):
    
    def __init__(self, D_in, D_out):
        super(CNNClassifier, self).__init__()
       
        # set dim. (constants)
        f_size = 5 #convolution filter size (kernel size)
        # p_size = 2  #max pooling filter size
        p_stride = 2 #conv/pooling stride
        p_padding = 1 #conv/pooling padding
        n_filter = 8 #output_size = x_h - n_filter + 1
        # fc_size = 16 * 8 * 8 #w/o pooling
        fc_size = 512 

        # 1) Conv
        self.conv_1 = nn.Conv2d(D_in, n_filter, f_size, stride=p_stride, padding=p_padding) 
        self.conv_2 = nn.Conv2d(8, 16, 3, stride=p_stride, padding=p_padding) # hyperparameters are calculated by the hyperparameters in the previous layer (CONV)
        self.conv_3 = nn.Conv2d(16, 32, 3, stride=p_stride, padding=p_padding) # hyperparameters are calculated by the hyperparameters in the previous layer (CONV)
        self.conv_4 = nn.Conv2d(32, 64, 3, stride=1) # hyperparameters are calculated by the hyperparameters in the previous layer (CONV)
        self.conv_5 = nn.Conv2d(64, 128, 3, stride=1, padding=p_padding) # hyperparameters are calculated by the hyperparameters in the previous layer (CONV)
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')

        # 2) Relu
        self.relu = nn.ReLU()  

        # 3) Pool
        # self.pool = nn.MaxPool2d(kernel_size = p_size, stride=1)
        # torch.nn.MaxPool2d(kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)

        # OPT: Batchnorm
        self.batch = nn.BatchNorm1d (fc_size)

        # 4) Linear 
        self.fc = nn.Linear(fc_size, D_out)          

        # Simple version 1~4 w/o pooling
        # self.cnn= nn.Sequential(
        #                     nn.Conv2d(3, 8, kernel_size=5, stride=2, padding=1),
        #                     nn.ReLU(),
        #                     nn.Conv2d(8, 16, kernel_size=3, stride=2, padding=1),
        #                     nn.ReLU()
        #             )
        # self.fc = nn.Linear(16 * 8 * 8, 10)       
        
        
    def forward(self, x):       
        # size of x: torch.Size([64, 3, 32, 32])
        
        #layer 1 Conv
        l1 = self.conv_1(x)
        # print("layer1-Conv:", l1.size()) 

        #layer 2 ReLU
        l2 = self.relu(l1)
        # print("layer2-ReLU:", l2.size()) 

        #layer 3 Conv
        l3 = self.conv_2(l2)
        # print("layer3-Conv:", l3.size()) 

        #layer 4 ReLU
        l4 = self.relu(l3)
        # print("layer4-ReLU:", l4.size()) 

        #layer 5 Conv
        l5 = self.conv_3(l4)
        # print("layer5-Conv:", l5.size()) 

        #layer 6 ReLU
        l6 = self.relu(l5)
        # print("layer6-ReLU:", l6.size()) 

        #layer 7 Conv
        l7 = self.conv_4(l6)
        # print("layer7-Conv:", l7.size()) 

        #layer 8 ReLU
        l8 = self.relu(l7)
        # print("layer8-ReLU:", l8.size()) 

        #layer 9 Conv
        l9 = self.conv_5(l8)
        # print("layer7-Conv:", l7.size()) 

        #layer 10 ReLU
        l_pool = self.relu(l9)
        # print("layer8-ReLU:", l8.size()) 

        #layer Pooling
        # l_pool = self.pool(l_conv)
        # print("layer-Pooling:", l_pool.size())       

        #layer fully-connected (linear) + OPT: Apply BatchNorm
        # flatten the output of your convolutional networks with view()
        l_pool = l_pool.view(l_pool.size(0),-1)
        # print("fully-input:", l7.size()) 
        l_bat = self.batch (l_pool)
        y_pred = self.fc(l_bat)
        # print("layer-fc:", y_pred.size()) 

        return y_pred
        
        ## Simple version 1~4 w/o pooling
        # batch_size = x.size(0)
        # return self.fc(self.cnn(x).view(batch_size, -1))
        
        

In [9]:
# Set hyperparameters
epoch =15
lr = 1e-3
n_input = 3072
n_classes = 10
batch_size = 64

# Load dataset
train_loader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

#set dim.
# x (N, C,H, W) # torch.Size([64, 3, 32, 32])
D_in = 3 #input channel
D_out = n_classes #number of classes

#init.
model = CNNClassifier(D_in, D_out)
model.train() ## Set the module in a training mode

# Set optimizer
# optim = torch.optim.SGD(model.parameters(), lr=lr)
optim = torch.optim.Adam(model.parameters(), lr=lr, eps=1e-2)
# optim = torch.optim.RMSprop(model.parameters(), lr=lr)
# optim = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, nesterov=True)
# optim = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
# optim = torch.optim.Adagrad(model.parameters(), lr=lr)
# optim = torch.optim.SGD(model.parameters(), lr=lr)

# Set loss function
criterion = nn.CrossEntropyLoss() #torch.nn.CrossEntropyLoss(weight=None, size_average=None, ignore_index=-100, reduce=None, reduction='mean')
# criterion = nn.BCELoss()

#train
for e in range(epoch):
  loss_epoch = 0
  
  for x, y in train_loader:
    optim.zero_grad()   
    y_pred = model(x)

    loss = criterion(y_pred, y)

    #update
    loss.backward()
    optim.step()
    loss_epoch += loss.item()

  print(f'Epcoh {e}: {loss_epoch}')

#evaluation (train/test accuracies)
model.eval()
train_acc = get_model_acc(model, train_loader)
test_acc = get_model_acc(model, test_loader)
print(f'Training accuracy: {train_acc}, Testing accuracy: {test_acc}')

Epcoh 0: 1224.3110374212265
Epcoh 1: 1026.344265460968
Epcoh 2: 934.7565296888351
Epcoh 3: 866.5753960609436
Epcoh 4: 814.7807305455208
Epcoh 5: 767.5877351164818
Epcoh 6: 729.54525411129
Epcoh 7: 694.6232128739357
Epcoh 8: 660.1824998259544
Epcoh 9: 630.0203550457954
Epcoh 10: 603.7329992055893
Epcoh 11: 575.5309460163116
Epcoh 12: 555.0155566036701
Epcoh 13: 537.4864779114723
Epcoh 14: 513.6218617260456
tensor(40248)
tensor(6548)
Training accuracy: 80.49600219726562, Testing accuracy: 65.4800033569336
